step 1: 从RGB空间转成YCbCr空间

step 2: 将其分到三通道Y Cb Cr三个色彩空间

step 3: 将三个通道通过CLAHE处理，并将三通道合并

step 4: 将合并后的图像转换回RGB色彩空间。

In [1]:
import cv2
import numpy as np
from pathlib import Path

In [2]:
def rgb_to_ycbcr(img_rgb: np.ndarray) -> np.ndarray:
    """
    img_rgb: (H,W,3), float32, [0,1]
    return:  (H,W,3), float32, [0,1] 大致
    """
    r = img_rgb[..., 0]
    g = img_rgb[..., 1]
    b = img_rgb[..., 2]

    y  = 0.299 * r + 0.587 * g + 0.114 * b
    cb = -0.168736 * r - 0.331264 * g + 0.5 * b + 0.5
    cr = 0.5 * r - 0.418688 * g - 0.081312 * b + 0.5

    ycbcr = np.stack([y, cb, cr], axis=-1)
    return ycbcr

In [3]:
def ycbcr_to_rgb(img_ycbcr: np.ndarray) -> np.ndarray:
    """
    img_ycbcr: (H,W,3), float32, [0,1] 大致
    return:    (H,W,3), float32, [0,1] 大致
    """
    y  = img_ycbcr[..., 0]
    cb = img_ycbcr[..., 1]
    cr = img_ycbcr[..., 2]

    cb_shift = cb - 0.5
    cr_shift = cr - 0.5

    r = y + 1.402 * cr_shift
    g = y - 0.344136 * cb_shift - 0.714136 * cr_shift
    b = y + 1.772 * cb_shift

    rgb = np.stack([r, g, b], axis=-1)
    return rgb

In [4]:
def clahe_ycbcr_numpy(
    img_bgr_uint8: np.ndarray,
    clip_limit: float = 2.0,
    tile_grid_size=(8, 8)
) -> np.ndarray:
    """
    对一张 BGR uint8 图像做：
    BGR -> RGB -> YCbCr -> 三通道 CLAHE -> 回 RGB -> 回 BGR

    参数
    ----
    img_bgr_uint8: (H,W,3), uint8, OpenCV 读进来的原图

    返回
    ----
    out_bgr_uint8: (H,W,3), uint8，处理后的图像
    """
    # BGR -> RGB
    img_rgb = cv2.cvtColor(img_bgr_uint8, cv2.COLOR_BGR2RGB)
    img_rgb_f = img_rgb.astype(np.float32) / 255.0  # [0,1]

    # RGB -> YCbCr
    ycbcr = rgb_to_ycbcr(img_rgb_f)

    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    ycbcr_eq = np.empty_like(ycbcr, dtype=np.float32)

    # 对 Y / Cb / Cr 三个通道分别做 CLAHE
    for c in range(3):
        ch = ycbcr[..., c]
        ch_uint8 = np.clip(ch * 255.0, 0, 255).astype(np.uint8)
        ch_eq_uint8 = clahe.apply(ch_uint8)
        ch_eq = ch_eq_uint8.astype(np.float32) / 255.0
        ycbcr_eq[..., c] = ch_eq

    # YCbCr -> RGB
    rgb_eq = ycbcr_to_rgb(ycbcr_eq)
    rgb_eq = np.clip(rgb_eq, 0.0, 1.0)
    rgb_eq_uint8 = (rgb_eq * 255.0 + 0.5).astype(np.uint8)

    # RGB -> BGR，方便 cv2.imwrite
    bgr_eq_uint8 = cv2.cvtColor(rgb_eq_uint8, cv2.COLOR_RGB2BGR)
    return bgr_eq_uint8

In [5]:
def process_folder(
    input_dir: str,
    output_dir: str,
    suffixes=(".png", ".jpg", ".jpeg", ".tif", ".tiff"),
    clip_limit: float = 2.0,
    tile_grid_size=(8, 8),
):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    img_paths = []
    for suf in suffixes:
        img_paths.extend(input_dir.rglob(f"*{suf}"))

    print(f"发现 {len(img_paths)} 张图片")

    for p in img_paths:
        img = cv2.imread(str(p), cv2.IMREAD_COLOR)  # BGR, uint8
        if img is None:
            print(f"[WARN] 读取失败: {p}")
            continue

        img_eq = clahe_ycbcr_numpy(
            img,
            clip_limit=clip_limit,
            tile_grid_size=tile_grid_size,
        )

        # 保存到输出目录，保持相对路径结构
        rel = p.relative_to(input_dir)
        out_path = output_dir / rel
        out_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(out_path), img_eq)

        print(f"处理并保存: {rel}")

In [6]:
process_folder(
    input_dir="../data/DRIVE/training/images",          # 原始图像根目录
    output_dir="data/ycbcr", # 处理后输出目录
    clip_limit=2.0,
    tile_grid_size=(8, 8),
)

发现 21 张图片
处理并保存: 26_training.tif
处理并保存: 25_training.tif
处理并保存: 28_training.tif
处理并保存: 31_training.tif
处理并保存: 38_training.tif
处理并保存: 30_training.tif
处理并保存: 40_training.tif
处理并保存: 23_training.tif
处理并保存: 22_training.tif
处理并保存: 21_training.tif
处理并保存: 36_training.tif
处理并保存: 27_training.tif
处理并保存: 35_training.tif
处理并保存: 37_training.tif
处理并保存: 32_training.tif
处理并保存: 29_training.tif
处理并保存: 34_training.tif
处理并保存: 33_training.tif
处理并保存: 39_training.tif
处理并保存: 24_training.tif
处理并保存: .ipynb_checkpoints/21_training-checkpoint.tif
